In [ ]:
!pip install skforecast
!pip install dask[dataframe]

In [ ]:
import pandas as pd
import lightgbm as lgb
import joblib
import matplotlib.pyplot as plt
import warnings

plt.style.use('fivethirtyeight')
plt.rcParams['lines.linewidth'] = 1.5
plt.rcParams['font.size'] = 10

from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom
from skforecast.ForecasterAutoregDirect import ForecasterAutoregDirect
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster
from skforecast.utils import save_forecaster
from skforecast.utils import load_forecaster
from skforecast.datasets import fetch_dataset

warnings.filterwarnings('once')

In [ ]:
model = joblib.load('ML models/LightGBM_ML.pkl')

In [ ]:
old_data = pd.read_csv('Fitabase Data 4.12.16-5.12.16/test_train_data.csv')
data = pd.read_csv('FitBit API Database/test_train_data_api_merged.csv')

print("Archivo : test_train_data.csv")
print(old_data)
print("\n")
print("Archivo : test_train_data_api_merged.csv")
print(data)

In [ ]:
#Comprobamos los valores NaN de cada columna
data.isna().sum()

In [ ]:
archivo_final = "FitBit API Database/test_train_data_api_merged.csv"
data = data.dropna(subset=['HeartRate'])
data = data.dropna(subset=['Steps'])
print(data)
data.to_csv(archivo_final, index=False)

In [ ]:
data = data.set_index('Time')
#data = data.asfreq(freq='60s')
data = data.sort_index()
                   
print(data.head())

In [ ]:
fig, ax = plt.subplots(figsize=(6, 2.5))
data['HeartRate'].plot(ax=ax, label='HeartRate')
ax.legend();

In [ ]:
#predecir para dentro de 5 minutos
steps = 5 

datos_train = data[:-steps]
datos_test  = data[-steps:]

print(
    f"Fechas train : {datos_train.index.min()} --- {datos_train.index.max()}  (n={len(datos_train)})"
)
print(
    f"Fechas test  : {datos_test.index.min()} --- {datos_test.index.max()}  (n={len(datos_test)})"
)


print(datos_train.head())

In [ ]:
forecaster = ForecasterAutoreg(
                regressor = model,
                lags      = 7 ,
                differentiation = None
             )

forecaster.fit(y=datos_train['HeartRate'], exog=datos_train[['Calories','Steps','Distance']])
forecaster

In [ ]:
predicciones = forecaster.predict(steps=5,  exog=datos_train[['Calories', 'Steps', 'Distance']])

In [ ]:
fig, ax = plt.subplots(figsize=(6, 2.5))
datos_train['HeartRate'].plot(ax=ax, label='train')
datos_test['HeartRate'].plot(ax=ax, label='test')
predicciones.plot(ax=ax, label='predicciones')
ax.legend();